In [77]:
import pandas as pd
import re
from tqdm import tqdm

from nltk.corpus import stopwords
from string import punctuation

import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize
# from razdel import tokenize

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oslikdau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## load functions

In [ ]:
def clean_text_old(newtext):
  newtext = str(newtext)
  stopwords = [
  "'type': 'bold'"
  ,"'type': 'italic'"
  , "'type': 'mention', 'text': '@lentadnya'"
  ,"'href': " #убрать весь адрес http
  ,"'type': 'text_link'"
  ,"'text':"
  ,"{" ,"}"
  ,', ,'
  , "[", "]"
  , "'"
  ]

  for stopword in stopwords:
    newtext = newtext.replace(stopword, "")

  replace_spaces =["\\n", "  ", ", ", "\\xa0"]
  for replace_space in replace_spaces:
    newtext = newtext.replace(replace_space, " ")

  # лишние пробелы похоже не влияют на предсказания модели. Зато эмоджи влияют. Оставляем только текстовые знаки и препинания.
  newtext = re.sub('[^А-Яа-яЁёA-Za-z0-9 _.,!-—?"«»]*', "", newtext)

  newtext = newtext.replace('ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'\
                            , '')

  return newtext

In [98]:
morph = pymorphy2.MorphAnalyzer()
ALLOWED_POS_TAGS = {'NOUN', 'ADJF', 'ADJS', 'VERB', 'INFN', 'PRTF', 'GRND', 'PRTS', 'ADVB'}

def clean_text(newtext):
  newtext = str(newtext)

  replace_spaces =["\\n", "  ", ", ", "\\xa0"]
  
  for replace_space in replace_spaces:
    newtext = newtext.replace(replace_space, " ")

  newtext = newtext.replace('ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'\
                            , '')

  current_tokens = simple_word_tokenize(newtext)
  current_pos_filtered_lemmata = []
  for token in current_tokens:
      parsed = morph.parse(token)[0]
      if parsed.tag.POS in ALLOWED_POS_TAGS:
          current_pos_filtered_lemmata.append(parsed.normal_form)

  newtext = ' '.join(current_pos_filtered_lemmata)

  return newtext

In [99]:
# def filter_war_texts(df_test):
#   df_war_news = df_test[
#         (df_test['text'].str.contains('воен|войн|спецоперац|обстрел|арм|СВО', case=False)) 
#         & ~(df_test['text'].str.contains('Главное к утру|Главные события|Главные новости', case=True)) #убираем сводки/дайджесты из нескольких новостей
#         & (df_test['text'].str.contains('укр|ВСУ|Азов', case=False))
#         ]
#   return df_war_news

def read_clean_json_news(file_path, date_from='2022-01-01'):
  # read json-file to dataframe
  df_js = pd.read_json(file_path)
  df_full = pd.DataFrame(df_js['messages'].tolist())
  df_full = df_full[['id', 'date', 'text']]
  df_full['date'] = pd.to_datetime(df_full['date'])
  # select specific dates
  df_test = df_full[df_full['date'] > date_from]
  # filter news by key words
  # df_test = filter_war_texts(df_test)
  # clean news texts and put into 'news' column
  df_test['news'] = df_test.apply(lambda row: clean_text(row['text']), axis=1)
  df_test.drop('text', axis=1, inplace=True)
  # clean of empty news
  df_test = df_test[df_test['news'].str.len()>5]
  return df_test

In [100]:
def filter_war_news(df_test):
  df_war_news = df_test[
        (df_test['news'].str.contains('воен|войн|спецоперац|обстрел|арм|СВО', case=False)) 
        & ~(df_test['news'].str.contains('Главное к утру|Главные события|Главные новости', case=True)) #убираем сводки/дайджесты из нескольких новостей
        & (df_test['news'].str.contains('укр|ВСУ|Азов', case=False))
        ]
  return df_war_news

## get texts ready

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
# 1) NEWS LOAD & PRER
# PARAMETERS
paths = [
        'news TG channels/meduza - from jan22.json',
        'news TG channels/rt.com - from jan22.json',
        'news TG channels/bbc - from jan22.json',
        'news TG channels/ria - from jan22.json'
        ]
date_from = '2022-02-24'

df_news_list = []
for path in paths:
  df_news_ = read_clean_json_news(path, date_from)
  name_position = path.find('channels/') + len('channels/')
  source_name = path[name_position:name_position+3]
  df_news_['source'] = source_name
  df_news_list.append(df_news_)

df_news = pd.concat(df_news_list, axis=0)

df_war_news = filter_war_news(df_news) # run this to filter by key words on war & remove digests

In [102]:
len(df_news), len(df_war_news)

(46820, 12727)

In [103]:
df_news.reset_index(inplace=True)
df_war_news.reset_index(inplace=True)

In [104]:
df_news.head(5)

,index,id,date,news,source
0,1875,51666,2022-02-24 01:11:48,пентагон считать российский войско прибывать т...,med
1,1876,51667,2022-02-24 01:19:32,украина запросить срочный заседание совбез оон...,med
2,1877,51668,2022-02-24 01:42:09,мобильный связь республиканский оператор феник...,med
3,1878,51669,2022-02-24 02:18:25,президент украина владимир зеленский выступить...,med
4,1879,51670,2022-02-24 02:20:21,санкция ес признание россия днр вступить сила ...,med


In [105]:
df_war_news.head(5)

,index,id,date,news,source
0,1876,51667,2022-02-24 01:19:32,украина запросить срочный заседание совбез оон...,med
1,1878,51669,2022-02-24 02:18:25,президент украина владимир зеленский выступить...,med
2,1881,51672,2022-02-24 03:11:18,полный расшифровка обращение президент украина...,med
3,1882,51673,2022-02-24 05:58:45,путин объявить начало военный операция донбасс...,med
4,1893,51684,2022-02-24 06:31:28,джо байден осудить действие россия слово амери...,med


In [106]:
df_war_news['news'][0]

'украина запросить срочный заседание совбез оон связь обращение днр рф просьба оказать военный помощь сообщить глава мид украина кулеб'

In [107]:
df_news.to_csv('df_news.csv')
df_war_news.to_csv('df_war_news.csv')

## BERTopic

In [108]:
# Topic model
from bertopic import BERTopic
# Dimension reduction
from umap import UMAP

import pandas as pd

### War news

In [110]:
df_news = pd.read_csv('df_news.csv', index_col=0)
df_war_news = pd.read_csv('df_war_news.csv', index_col=0)

In [111]:
df_news.shape, df_war_news.shape

((46820, 5), (12727, 5))

In [112]:
df_war_news

,index,id,date,news,source
0,1876,51667,2022-02-24 01:19:32,украина запросить срочный заседание совбез оон...,med
1,1878,51669,2022-02-24 02:18:25,президент украина владимир зеленский выступить...,med
2,1881,51672,2022-02-24 03:11:18,полный расшифровка обращение президент украина...,med
3,1882,51673,2022-02-24 05:58:45,путин объявить начало военный операция донбасс...,med
4,1893,51684,2022-02-24 06:31:28,джо байден осудить действие россия слово амери...,med
...,...,...,...,...,...
12722,27477,165025,2022-05-27 12:44:00,называть общий сумма ущерб днр преждевременно ...,ria
12723,27478,165026,2022-05-27 12:51:54,песок прокомментировать утверждение идея джонс...,ria
12724,27483,165031,2022-05-27 13:25:20,бастрыкин мариуполь провести совещание штаб ра...,ria
12725,27492,165040,2022-05-27 14:16:48,украинский военный отступать подорвать перепра...,ria


In [113]:
# 13K mews = 26min
# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="russian", calculate_probabilities=True)
# Run BERTopic model
topics, probabilities = topic_model.fit_transform(df_war_news['news'])

In [115]:
# topics
df_war_news['topic'] = topics

In [116]:
df_war_news.to_excel('results/df_war_news_w_topics.xls')

In [117]:
# Topics by sources
# topic_by_source_perc = pd.crosstab(df_war_news['topic'], df_war_news['source']
#             , margins=True
#             , normalize='columns'
#             ).sort_values('All', ascending=False)

# absolutes
topic_by_source_abs = pd.crosstab(df_war_news['topic'], df_war_news['source']
            , margins=True
            # , normalize='columns'
            ).sort_values('All', ascending=False)

In [118]:
topic_by_source_abs.drop('All', axis=0, inplace=True)

In [119]:
topic_by_source_abs

source,bbc,med,ria,rt.,All
topic,,,,,
-1,1400,1421,982,2082,5885
0,105,94,78,105,382
1,74,94,72,98,338
2,87,79,47,108,321
3,112,105,13,38,268
...,...,...,...,...,...
135,3,4,1,3,11
136,4,3,2,2,11
137,1,0,1,9,11


In [120]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,5885,-1_украинский_украина_быть_россия
1,0,382,0_путин_кремль_президент_владимир
2,1,338,1_суд_преступление_уголовный_дело
3,2,321,2_рф_человек_быть_город
4,3,268,3_направление_генштаб_наступление_сводка
...,...,...,...
136,135,11,135_вышемирской_роддом_девушка_ролик
137,136,11,136_турецкий_турция_корабль_калин
138,137,11,137_вручить_орден_медаль_мужество
139,138,11,138_рота_противник_националист_лейтенант


In [121]:
topic_by_source_abs = pd.merge(topic_by_source_abs, topic_model.get_topic_info()[['Topic','Name']], left_index=True, right_on='Topic')

In [122]:
topic_by_source_abs.to_excel('results/sources_by_topics_abs.xls')

In [126]:
topic_by_source_abs.head(3)

,bbc,med,ria,rt.,All,Topic,Name
0,1400,1421,982,2082,5885,-1,-1_украинский_украина_быть_россия
1,105,94,78,105,382,0,0_путин_кремль_президент_владимир
2,74,94,72,98,338,1,1_суд_преступление_уголовный_дело


In [127]:
topic_by_source_abs['alt'] = topic_by_source_abs[['bbc','med']].sum(axis=1)
topic_by_source_abs['prop'] = topic_by_source_abs[['ria','rt.']].sum(axis=1)
topic_by_source_abs['prop_share'] = topic_by_source_abs['prop'] / topic_by_source_abs['All']

In [135]:
threshold = 0.3
min_sample = 40
topic_by_source_abs[((topic_by_source_abs['prop_share'] < threshold) | (topic_by_source_abs['prop_share'] > 1-threshold)) & (topic_by_source_abs['All']>=min_sample)].sort_values('prop_share')

,bbc,med,ria,rt.,All,Topic,Name,alt,prop,prop_share
45,24,14,3,1,42,44,44_оон_погибнуть_жертва_мирный,38,4,0.095238
13,54,42,2,9,107,12,12_беженец_миллион_млн_тысяча,96,11,0.102804
32,45,5,3,3,56,31,31_подкаст_день_война_любимый,50,6,0.107143
17,68,13,2,9,92,16,16_рубль_банк_валюта_компания,81,11,0.119565
8,65,61,11,11,148,7,7_потеря_погибнуть_би_гибель,126,22,0.148649
41,32,6,4,3,45,40,40_час_главное_день_российский,38,7,0.155556
23,29,34,4,8,75,22,22_ребёнок_погибнуть_ранение_пострадать,63,12,0.160000
29,44,8,7,4,63,28,28_британский_разведка_британия_великобритания,52,11,0.174603
18,33,41,12,5,91,17,17_компания_приостановить_бизнес_россия,74,17,0.186813
4,112,105,13,38,268,3,3_направление_генштаб_наступление_сводка,217,51,0.190299


In [154]:
topic_by_source_abs[(topic_by_source_abs['All']>=min_sample) & topic_by_source_abs['Name'].str.contains('сдать')]

,bbc,med,ria,rt.,All,Topic,Name,alt,prop,prop_share
36,0,0,15,37,52,35,35_бросить_сдаться_командование_плен,0,52,1.0


In [54]:
len(set(topics))

160

In [146]:
# Get top 10 terms for a topic
topic_model.get_topic(20)

[('военкор', 0.044972032782373425),
 ('азовсталь', 0.04228978100273216),
 ('азов', 0.03158613500551794),
 ('андрей', 0.030151259502367387),
 ('боевик', 0.027411520370812053),
 ('филатов', 0.027269867615297316),
 ('завод', 0.025336089387980834),
 ('азовец', 0.02452365474213358),
 ('комбинат', 0.02407125127971125),
 ('наш', 0.02252599784685213)]

In [151]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=12)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.009587114067917893, 0.01031389475587975,
                    0.01352008716047543, 0.013803166776060414, 0.04118858309205932],
              'xaxis': 'x',
              'y': [песок  , владимир  , президент  , кремль  , путин  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.014839396432620744, 0.017281301664721995,
                    0.022022845033567502, 0.026512456492350397,
                    0.02809952378051308],
              'xaxis': 'x2',
              'y': [расследование  , дело  , уголовный  , преступление  , суд  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.005772380833273076, 0.005833303057341011,
                    0.00607865744057563, 0.006374478942245669,
                    0.007987457191292562],
              'xaxis': 'x3',
              'y': [русский  , город  , быть  , человек  , рф  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.015212353146813434, 0.01614820892838099,
                    0.017064653517186036, 0.019509697794556705,
                    0.028842746047188197],
              'xaxis': 'x4',
              'y': [донецкий  , сводка  , наступление  , генштаб  , направление  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.021703175614279105, 0.02174275595529699,
                    0.024577324331710785, 0.03147481288924172, 0.04948848449417033],
              'xaxis': 'x5',
              'y': [блокировка  , удалить  , сайт  , заблокировать  , роскомнадзор
                    ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.009679921911912297, 0.009692749139242264,
                    0.013682552494856547, 0.019782153732107857,
                    0.04935371804642743],
              'xaxis': 'x6',
              'y': [украина  , сказать  , владимир  , президент  , зеленский  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.025202229603210377, 0.025486354672241692,
                    0.03727407097629479, 0.03789421934392114, 0.038200376427910625],
              'xaxis': 'x7',
              'y': [млн  , пакет  , помощь  , доллар  , млрд  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.014617936343533028, 0.017651138296070006,
                    0.020333362942824895, 0.03570054948442548,
                    0.053358260089367525],
              'xaxis': 'x8',
              'y': [потерять  , гибель  , би  , погибнуть  , потеря  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.013912214149110515, 0.014110018189404019,
                    0.016977414634410295, 0.021145311419722805,
                    0.05222886920118737],
              'xaxis': 'x9',
              'y': [раунд  , подоляк  , мединский  , делегация  , переговоры  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04971274748009167, 0.05300582492852311,
                    0.054567269572288524, 0.06766365340384585, 0.08747247697561535],
              'xaxis': 'x10',
              'y': 

### Navalny news

In [156]:
df_news = pd.read_csv('df_news.csv', index_col=0)

def filter_navalny(df_test):
  df_filtered_news = df_test[
        (df_test['news'].str.contains('Навальн', case=False)) 
        & ~(df_test['news'].str.contains('Главное к утру|Главные события|Главные новости', case=True)) #убираем сводки/дайджесты из нескольких новостей
        # & (df_test['news'].str.contains('укр|ВСУ|Азов', case=False))
        ]
  return df_filtered_news

df_naval_news = filter_navalny(df_news) # run this to filter by key words on war & remove digests

In [162]:
df_naval_news.reset_index(inplace=True)

In [163]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Initiate BERTopic
topic_model1 = BERTopic(umap_model=umap_model, language="russian", calculate_probabilities=True)
# Run BERTopic model
topics1, probabilities1 = topic_model1.fit_transform(df_naval_news['news'])

In [164]:
# topics
df_naval_news['topic'] = topics1

In [165]:
df_naval_news.to_excel('results/df_naval_news_w_topics.xls')

In [166]:
# Topics by sources
# topic_by_source_perc = pd.crosstab(df_war_news['topic'], df_war_news['source']
#             , margins=True
#             , normalize='columns'
#             ).sort_values('All', ascending=False)

# absolutes
topic_by_source_abs_nav = pd.crosstab(df_naval_news['topic'], df_naval_news['source']
            , margins=True
            # , normalize='columns'
            ).sort_values('All', ascending=False)

In [167]:
topic_by_source_abs_nav.drop('All', axis=0, inplace=True)

In [168]:
topic_by_source_abs_nav = pd.merge(topic_by_source_abs_nav, topic_model1.get_topic_info()[['Topic','Name']], left_index=True, right_on='Topic')

In [169]:
topic_by_source_abs_nav.to_excel('results/topic_by_source_abs_nav.xls')

In [175]:
topic_by_source_abs_nav

,bbc,med,ria,rt.,All,Topic,Name,alt,prop,prop_share
1,29,43,1,18,91,0,0_навальный_быть_россия_который,72,19,0.208791
2,10,15,20,14,59,1,1_навальный_суд_год_колония,25,34,0.576271
0,4,6,0,2,12,-1,-1_навальный_суд_алехин_насилие,10,2,0.166667


In [171]:
topic_by_source_abs_nav['alt'] = topic_by_source_abs_nav[['bbc','med']].sum(axis=1)
topic_by_source_abs_nav['prop'] = topic_by_source_abs_nav[['ria','rt.']].sum(axis=1)
topic_by_source_abs_nav['prop_share'] = topic_by_source_abs_nav['prop'] / topic_by_source_abs_nav['All']

In [174]:
threshold = 0.5
min_sample = 10
topic_by_source_abs_nav[((topic_by_source_abs_nav['prop_share'] < threshold) | (topic_by_source_abs_nav['prop_share'] > 1-threshold)) & (topic_by_source_abs_nav['All']>=min_sample)].sort_values('prop_share')

,bbc,med,ria,rt.,All,Topic,Name,alt,prop,prop_share
0,4,6,0,2,12,-1,-1_навальный_суд_алехин_насилие,10,2,0.166667
1,29,43,1,18,91,0,0_навальный_быть_россия_который,72,19,0.208791
2,10,15,20,14,59,1,1_навальный_суд_год_колония,25,34,0.576271


In [216]:
topic_model1.get_topic(-1)

[('навальный', 0.07606961218422634),
 ('суд', 0.06397953670670499),
 ('алехин', 0.06152497775688406),
 ('насилие', 0.058428510801733224),
 ('год', 0.050582660244699755),
 ('заблокировать', 0.043187142837730015),
 ('требование', 0.043187142837730015),
 ('страница', 0.040202375875328114),
 ('процесс', 0.03984036631978142),
 ('дело', 0.039611538943944445)]

In [234]:
print('\n NEW \n'.join(df_naval_news[df_naval_news['topic'] == 1]['news']))

пресс-секретарь навальный кира ярмыш объявить розыск попросить заменить ограничение свобода санитарный дело реальный срок обнаружить лето прошлое год ярмыш приговорить год ограничение свобода санитарный дело покинуть россия
 NEW 
гособвинение требовать приговорить алексей навальный год колония штраф млн тыс дело мошенничество оскорбление суд сообщать корреспондент новый газета
 NEW 
гособвинение уточнять требовать навальный год строгий режим
 NEW 
приговор алексей навальный огласить март политик выступить суд последний слово передавать корреспондент новый
 NEW 
судья рассматривать дело навальный повысить указ президент судья лефортовский районный суд маргарита котов назначить судья мосгорсуд уход новый пост должный быть завершить текущий дело тот число вынести приговор навальный
 NEW 
покровский колония начинаться оглашение приговор алексей навальный фото андрей карев новый газета
 NEW 
алексей навальный адвокат ольга михайлов вадим кобзев оглашение приговор дело мошенничество оскорбле

### Mariupol news

In [212]:
df_news['news'].str.contains('главный утро|главный событие|главный новость').sum()

153

In [214]:
df_news[(df_news['news'].str.contains('мариупол|азовстал')) & ~(df_news['news'].str.contains('главный утро|главный событие|главный новость'))]

,index,id,date,news,source
20,1895,51686,2022-02-24 06:38:06,соцсеть публиковать видео стрельба российский ...,med
31,1906,51697,2022-02-24 07:03:15,взрыв весь приграничный город украина киев оде...,med
85,1961,51752,2022-02-24 10:54:47,мвд украина сообщать погибнуть результат обстр...,med
110,1986,51777,2022-02-24 12:56:15,генштаб сообщить возвращение контроль мариупол...,med
168,2047,51841,2022-02-24 17:51:04,мариуполь говорить русский украинский злой объ...,med
...,...,...,...,...,...
46767,27439,164987,2022-05-27 09:01:01,порт мариуполь быть задействовать полный объём...,ria
46769,27441,164989,2022-05-27 09:09:25,пансионат скадовск херсонский область готовый ...,ria
46770,27442,164990,2022-05-27 09:15:20,днр планировать восстановить аэропорт донецк м...,ria
46775,27447,164995,2022-05-27 09:47:30,житель мариуполь хотеть восстанавливать завод ...,ria


### Bucha news

In [215]:
df_news[(df_news['news'].str.contains('буча')) & ~(df_news['news'].str.contains('главный утро|главный событие|главный новость'))]

,index,id,date,news,source
343,2223,52018,2022-02-25 16:58:21,война украина второй день анализировать ситуац...,med
626,2538,52338,2022-02-27 09:49:28,город буча километр киев трасса заметить росси...,med
633,2545,52345,2022-02-27 10:58:18,данные украинский гсчс результат попадание сна...,med
700,2622,52422,2022-02-27 17:21:53,радио свобода аэропорт гостомель киев быть сже...,med
1004,2929,52737,2022-03-01 10:03:14,итог пятый день война украина тяжёлый артиллер...,med
...,...,...,...,...,...
44646,25153,162689,2022-05-10 22:05:46,76-й день спецоперация россия украина главное ...,ria
45111,25663,163202,2022-05-13 19:20:57,сотня житель мали выйти акция протест потребов...,ria
45147,25701,163240,2022-05-14 10:10:00,глава французский гуманитарный ассоциация помо...,ria
46412,27061,164608,2022-05-25 09:56:52,техасский резня такой заголовок трельба америк...,ria
